In [1]:
import rclpy
from rclpy.node import Node
from rclpy.action import ActionClient
import gc

from std_msgs.msg import String
from geometry_msgs.msg import Pose
from geographic_msgs.msg import GeoPoint
from lotusim_msgs.msg import MASCmd as MASCmdMsg, VesselPositionArray
from lotusim_msgs.action import MASCmd, MASCmdArray

The messages (string,MASCmd,VesselPositionArray) are in the interface folder. Please open it. 

In MASCmd, the message is divided into goal, result, feedback while its running. We are mainly interested in the goal and result msg 
```
std_msgs/Header header
MASCmd cmd
---
bool result
string name
uint16 entity
---
bool fb
```

Each message is treated like a class variable. To access the MASCmd under goal, use `msg.cmd` too access.

Below are example of publisher, subscriber and action server.

In [ ]:
class ExampleNode(Node):
    def __init__(self):
        super().__init__('example_node')
        self.pose_subscription = self.create_subscription(
            VesselPositionArray,
            'defenseScenario/poses',
            self.poses_callback,
            10)
        self.random_publisher = self.create_publisher(String, 'topic', 10)
        self.mas_action_client = ActionClient(self, MASCmd, '/defenseScenario/mas_cmd')
        self.mas_array_action_client = ActionClient(self, MASCmdArray, '/defenseScenario/mas_cmd_array')
        self.vessel_poses= {}

    def poses_callback(self, msg):
        for vessel_position in msg.vessels:
            lat = vessel_position.geo_point.latitude
            long = vessel_position.geo_point.longitude
            self.vessel_poses[vessel_position.vessel_name] = (lat, long)

    def send_rando_string(self):
        msg = String()
        self.random_publisher.publish(msg)
    
    def spawn_ship_with_dynamics(self,id):
        msg = MASCmdMsg()
        msg.cmd_type =  MASCmdMsg.CREATE_CMD
        msg.model_name =  "lrauv"
        msg.vessel_name =  "lrauv_" + str(id)        
        latlong_msg = GeoPoint()
        latlong_msg.latitude = 1.2605794416293148
        latlong_msg.longitude = 103.7516212463379
        latlong_msg.altitude = -10.0
        msg.geo_point = latlong_msg
        
        msg.sdf_string = """
        <lotus_param>
            <render_interface>
                <publish_render>true</publish_render>
                <renderer_type_name>lrauv</renderer_type_name>
            </render_interface>
            <physics_engine_interface>
            <underwater>
                <connection_type>XDynWebSocket</connection_type>
                <uri>ws://127.0.0.1:12346</uri>
                <thrusters>
                    <thrusters1>thruster1</thrusters1>
                </thrusters>
            </underwater>
            <surface>
                <connection_type>XDynWebSocket</connection_type>
                <uri>ws://127.0.0.1:12345</uri>
                <thrusters>
                    <thrusters1>thruster1</thrusters1>
                </thrusters>
            </surface>
            <init_state>Underwater</init_state>
            </physics_engine_interface>
        </lotus_param>
        """


    def spawn_aerial_drone(self,id):
        msg = MASCmdMsg()
        msg.cmd_type =  MASCmdMsg.CREATE_CMD
        msg.model_name =  "x500"
        msg.vessel_name =  "x500_" + str(id)        
        latlong_msg = GeoPoint()
        latlong_msg.latitude = 1.2605794416293148
        latlong_msg.longitude = 103.7516212463379
        latlong_msg.altitude = 10.0
        msg.geo_point = latlong_msg
        
        msg.sdf_string = """
        <lotus_param>
            <render_interface>
                <publish_render>true</publish_render>
                <renderer_type_name>lrauv</renderer_type_name>
            </render_interface>
            <physics_engine_interface>
                  <aerial>
                    <connection_type>ROS2</connection_type>
                    <namespace>aerialWorld</namespace>
                  </aerial>
                  <init_state>Aerial</init_state>
            </physics_engine_interface>
        </lotus_param>
        """
        goal_msg = MASCmd.Goal()
        goal_msg.cmd = msg
        self.mas_action_client.wait_for_server()
        return self.mas_action_client.send_goal_async(goal_msg)

    def spawn_circling_ship(self, id):
        goal_msg = MASCmd.Goal()
       
        msg = MASCmdMsg()
        msg.cmd_type =  MASCmdMsg.CREATE_CMD
        msg.model_name =  "dtmb_hull"
        msg.vessel_name =  "dtmb_" + str(id)        
        latlong_msg = GeoPoint()
        latlong_msg.latitude = 1.2605794416293148
        latlong_msg.longitude = 103.7516212463379
        latlong_msg.altitude = 0.0
        msg.geo_point = latlong_msg
        
        msg.sdf_string = """
        <lotus_param>
            <waypoint_follower>
                <follower>
                    <loop>true</loop>
                    <linear_accel_limit>0.5</linear_accel_limit>
                    <angular_accel_limit>0.005</angular_accel_limit>
                    <angular_velocities_limits>0.01</angular_velocities_limits>
                    <range_tolerance>2</range_tolerance>
                    <circle>
                        <radius>10</radius>
                    </circle>
                </follower>
            </waypoint_follower>
        </lotus_param>
        """

        goal_msg.cmd = msg
        self.mas_action_client.wait_for_server()
        return self.mas_action_client.send_goal_async(goal_msg)

    def spawn_multiple_circling_ship(self,number_of_ships):
        goal_msg = MASCmdArray.Goal()

        count = 0
        while (count <number_of_ships):
            msg = MASCmdMsg()
            msg.cmd_type =  MASCmdMsg.CREATE_CMD
            msg.model_name =  "dtmb_hull"
            msg.vessel_name =  "dtmb" + str(count)
            count += 1      
            latlong_msg = GeoPoint()
            latlong_msg.latitude = 1.2605794416293148
            latlong_msg.longitude = 103.7516212463379
            latlong_msg.altitude = 0.0
            msg.geo_point = latlong_msg
            
            msg.sdf_string = """
            <lotus_param>
                <waypoint_follower>
                    <follower>
                        <loop>true</loop>
                        <linear_accel_limit>0.5</linear_accel_limit>
                        <angular_accel_limit>0.005</angular_accel_limit>
                        <angular_velocities_limits>0.01</angular_velocities_limits>
                        <range_tolerance>2</range_tolerance>
                        <circle>
                            <radius>10</radius>
                        </circle>
                    </follower>
                </waypoint_follower>
            </lotus_param>
            """
            goal_msg.cmd.append(msg)
        self.mas_array_action_client.wait_for_server()
        return self.mas_array_action_client.send_goal_async(goal_msg)

    def get_vessel_poses(self):
        rclpy.spin_once(self)
        return self.vessel_poses

def main(args=None):
    if not rclpy.ok(): 
        rclpy.init(args=args)

    node = ExampleNode()
    node.send_mas_array()

    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        node.get_logger().info("Shutting down...")
    finally:
        node.destroy_node()
        rclpy.shutdown()

In [3]:
if not rclpy.ok(): 
    rclpy.init(args=None)

try:
    node.destroy_node()
    del node
    gc.collect()
except NameError:
    pass 
except Exception as e:
    print(f"Could not destroy node: {e}")


node = ExampleNode()


[WARN] [1762602116.795997005] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.


In [4]:
node.spawn_ship_with_dynamics(1)
